In [9]:
%load_ext sql

%sql sqlite:///publications.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @publications.db'

# Challenge 1 - Most Profiting Authors

## Step 1: Calculate the royalties of each sales for each author

In [10]:
%%sql

SELECT titleauthor.title_id, titleauthor.au_id, sales.qty, 
titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100 as sales_royalty 

FROM titleauthor INNER JOIN sales ON titleauthor.title_id = sales.title_id
INNER JOIN titles ON titleauthor.title_id = titles.title_id

ORDER BY au_id

 * sqlite:///publications.db
Done.


title_id,au_id,qty,sales_royalty
PS3333,172-32-1176,15,29.984999999999996
BU1032,213-46-8915,5,3.9979999999999993
BU1032,213-46-8915,10,7.995999999999999
BU2075,213-46-8915,35,25.116000000000003
PC1035,238-95-7766,30,110.16
BU1111,267-41-2394,25,11.95
TC7777,267-41-2394,20,8.994
BU7832,274-80-9391,15,29.984999999999996
BU1032,409-56-7008,5,5.996999999999999
BU1032,409-56-7008,10,11.993999999999998


## Step 2: Aggregate the total royalties for each title for each author

In [11]:
%%sql

SELECT titleauthor.title_id, titleauthor.au_id, sum(titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100) as sales_royalty 

FROM titleauthor INNER JOIN sales ON titleauthor.title_id = sales.title_id
INNER JOIN titles ON titleauthor.title_id = titles.title_id

GROUP BY titleauthor.title_id, titleauthor.au_id
ORDER BY au_id

 * sqlite:///publications.db
Done.


title_id,au_id,sales_royalty
PS3333,172-32-1176,29.984999999999996
BU1032,213-46-8915,11.993999999999998
BU2075,213-46-8915,25.116000000000003
PC1035,238-95-7766,110.16
BU1111,267-41-2394,11.95
TC7777,267-41-2394,8.994
BU7832,274-80-9391,29.984999999999996
BU1032,409-56-7008,17.990999999999996
PC8888,427-17-2319,50
TC7777,472-27-2349,8.994


## Step 3: Calculate the total profits of each author

In [12]:
%%sql


SELECT AUTHOR_ID, SUM(PROFIT) as PROFIT

FROM 
(SELECT  titleauthor.au_id as AUTHOR_ID, sum(titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100) +
        titles.advance* titleauthor.royaltyper / 100 as PROFIT 

FROM titleauthor INNER JOIN sales ON titleauthor.title_id = sales.title_id
INNER JOIN titles ON titleauthor.title_id = titles.title_id

GROUP BY titleauthor.title_id, titleauthor.au_id
ORDER BY au_id)

GROUP BY AUTHOR_ID
ORDER BY PROFIT DESC

LIMIT 3

 * sqlite:///publications.db
Done.


AUTHOR_ID,PROFIT
213-46-8915,12162.11
722-51-5454,11271.528
998-72-3567,7224.956


# Challenge 2

In [40]:
%%sql

CREATE TABLE 'most_profiting_authors' AS
    
    SELECT AUTHOR_ID, SUM(PROFIT) as PROFIT

    FROM 
    (SELECT  titleauthor.au_id as AUTHOR_ID, sum(titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100) +
            titles.advance* titleauthor.royaltyper / 100 as PROFIT 

    FROM titleauthor INNER JOIN sales ON titleauthor.title_id = sales.title_id
    INNER JOIN titles ON titleauthor.title_id = titles.title_id
    GROUP BY titleauthor.title_id, titleauthor.au_id
    ORDER BY au_id)

    GROUP BY AUTHOR_ID
    ORDER BY PROFIT DESC

 * sqlite:///publications.db
Done.


ResourceClosedError: This result object does not return rows. It has been closed automatically.